In [ ]:
import os, glob, platform, datetime, random
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.utils.data as data_utils
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.autograd import Variable
from torch import functional as F
# import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

import cv2
from PIL import Image
from tensorboardX import SummaryWriter

import numpy as np
from numpy.linalg import inv as denseinv
from scipy import sparse
from scipy.sparse import lil_matrix, csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import inv as spinv
import scipy.misc

# from myimagefolder import MyImageFolder
from mymodel import GradientNet
from myargs import Args

In [ ]:
def loadimg(path):
    im = Image.open(path).convert('RGB')
    w0,h0 = im.size
    w1,h1 = w0//32*32,h0//32*32
    print(im.size)
    print(w1,h1)
    im = transforms.ToTensor()(im)
    x = torch.zeros(1,3,h1,w1)
    x[0,:,:,:] = im[:,0:h1,0:w1]
    x = Variable(x, volatile=True)
    return x

In [ ]:
type_ = 'albedo'
gpu_num = 1
gradient = False
type2 = 'rgb' if gradient == False else 'gd'
image_slpit = True
input_img_paths = glob.glob('/media/lwp/xavier/tmp/test_image/*.jpg')
# root = '/media/lwp/xavier/graduation_results/showcase_model/image_split/{}/rgb/'.format(type_)
root = '/media/albertxavier/data/eccv/graduation-project/pytorch/snapshot0/'
snapshot = torch.load(root+'snapshot-418.pth.tar')
state_dict = snapshot['state_dict']
args = snapshot['args']
densenet = models.__dict__[args.arch](pretrained=True).cuda(gpu_num)
        
net = GradientNet(densenet=densenet, growth_rate=32, 
              transition_scale=2, pretrained_scale=4,
             gradient=gradient).cuda(gpu_num)
net.load_state_dict(state_dict)
net.train()
for input_img_path in input_img_paths:
    print(input_img_path)
    sp = input_img_path.split('.')
    output_img_path = os.path.join(sp[0]+type_+'.'+sp[1])
    im = loadimg(input_img_path).cuda(gpu_num)
    res = net(im.cuda(gpu_num), go_through_merge=True)
    merged = res[:,0:3,:,:]
    merged = merged[0]
    merged = merged.cpu().data.numpy()
    merged = merged.transpose(1,2,0)
    alpha = res[]

In [ ]:
# if gradient == False:
#     merged = mergeRGB[5]
#     merged = merged[0]
#     merged = merged.cpu().data.numpy()
#     print (merged.shape)
#     merged = merged.transpose(1,2,0)
#     print (merged.shape)
#     dx = merged[:,:,0:3]
#     cv2.imwrite('out_merge.png', dx[:,:,::-1]*255)


In [ ]:
# if gradient == True:
#     merged = mergeRGB[5]
#     merged = merged[0]
#     merged = merged.cpu().data.numpy()
#     print (merged.shape)
#     merged = merged.transpose(1,2,0)
#     print (merged.shape)
#     dy = merged[:,:,0:3]+0.5
#     dx = merged[:,:,3:6]+0.5
#     cv2.imwrite('out_merge_dx.png', dx[:,:,::-1]*255)
#     cv2.imwrite('out_merge_dy.png', dy[:,:,::-1]*255)
